# Pipeline
## Setting up
### set module paths and data path

In [ ]:
minian_path = "."
caiman_path = "."
dpath = "./demo_movies/"
meta_dict={'session_id': -1, 'session': -2, 'animal': -3}

### load modules

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(minian_path)
sys.path.append(caiman_path)
import gc
import psutil
import numpy as np
import xarray as xr
import holoviews as hv
import functools as fct
import paramnb
import caiman as cm
import matplotlib.pyplot as plt
import bokeh.plotting as bpl
import dask.array as da
import minian.visualization_ply as mvis
from bokeh.io import output_notebook, show
from bokeh.layouts import layout
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from IPython.core.display import display, HTML
from ipyparallel import Client
from minian.utilities import load_videos, varray_to_tif, save_cnmf, save_movies, scale_varr, save_varr
from minian.preprocessing import remove_background, detect_brightspot, correct_brightspot, gaussian_blur
from minian.motion_correction import estimate_shift_fft, apply_shifts, interpolate_frame
from minian.visualization import VArrayViewer, MCViewer, CNMFViewer
from minian.caiman_patch import local_correlations_fft, correlation_pnr
from caiman.cluster import setup_cluster
from caiman.source_extraction import cnmf
from caiman.utils.visualization import inspect_correlation_pnr, nb_view_patches
from caiman.components_evaluation import estimate_components_quality_auto

### module initialization

In [ ]:
cm.summary_images.local_correlations_fft = local_correlations_fft
cm.summary_images.correlation_pnr = correlation_pnr
dpath = os.path.abspath(dpath)
hv.notebook_extension('bokeh', width=100)

## Pre-processing
### loading videos and visualization

In [ ]:
%%time
varr = load_videos(dpath)

In [ ]:
%%output size=100 fps=30
vaviewer = VArrayViewer([varr], framerate=30)
display(vaviewer.widgets)
vaviewer.show()

### background removal

In [ ]:
%%time
varr_ref = remove_background(varr)

### bright spots removal

In [ ]:
%%time
spots = detect_brightspot(varr_ref, thres=4)
varr_ref = correct_brightspot(varr_ref, spots)
outliers = varr_ref > varr_ref.quantile(0.9999999)
varr_ref = correct_brightspot(varr_ref, outliers)

### gaussian blur

In [ ]:
%%time
varr_ref = gaussian_blur(varr_ref)

### normalization

In [ ]:
%%time
varr_ref = scale_varr(varr_ref, (0, 255)).astype(np.uint8, copy=False)

### visualization of pre-processing

In [ ]:
%%output size=70 fps=30
vaviewer = VArrayViewer([varr, varr_ref], framerate=30)
display(vaviewer.widgets)
vaviewer.show()

## motion correction
### estimate shifts

In [ ]:
%%time
shifts, corr, mask = estimate_shift_fft(varr_ref, z_thres=None)

### apply shifts

In [ ]:
%%time
varr_mc, shifts_final = apply_shifts(varr_ref, shifts)

### interpolation

In [ ]:
%%time
varr_mc_int = interpolate_frame(varr_mc, mask)

### visualization of motion-correction

In [ ]:
%%output size=55 fps=10
vaviewer = VArrayViewer([varr, varr_mc, varr_mc_int], framerate=10)
display(vaviewer.widgets)
vaviewer.show()

### visualization of shifts

In [ ]:
%%output size=100
%%opts Curve [width=1500, tools=['hover']]
hv.NdOverlay(dict(width=hv.Curve(shifts.sel(shift_dim='width')), height=hv.Curve(shifts.sel(shift_dim='height'))))

### save result as tif

In [ ]:
%%time
varray_to_tif(dpath + os.sep + "varr_mc_int.tif", varr_mc_int)

### save result as DataSet

In [ ]:
%%time
varr = save_varr(varr_mc_int, dpath, meta_dict=meta_dict)

## CNMF
### save data as memmap

In [ ]:
fname = cm.save_memmap([dpath + os.sep + "varr_mc_int.tif"], base_name='varr_mc_int', order='F')
# fname = "./demo_movies/varr_mc_int_d1_480_d2_752_d3_1_order_F_frames_1000_.mmap"

### set up and load data

In [ ]:
os.chdir(caiman_path)
try:
    dview.terminate() # stop it if it was running
except:
    pass
c, dview, n_processes = setup_cluster(
    backend='local', # use this one
    n_processes=16,  # number of process to use, if you go out of memory try to reduce this one
)

In [ ]:
Yr, dims, T = cm.load_memmap(fname)
Y = Yr.T.reshape((T,) + dims, order='F')

### cnmf update

In [ ]:
cnm = cnmf.CNMF(n_processes=n_processes, 
                method_init='greedy_roi',               # use this for 1 photon
                k=15,                                   # neurons per patch
                gSig=(5, 5),                            # half size of neuron
                merge_thresh=.8,                        # threshold for merging
                p=1,                                    # order of autoregressive process to fit
                dview=dview,                            # if None it will run on a single thread
                tsub=2,                                 # downsampling factor in time for initialization, increase if you have memory problems             
                ssub=2,                                 # downsampling factor in space for initialization, increase if you have memory problems
                Ain=None,                               # if you want to initialize with some preselcted components you can pass them here as boolean vectors
                rf=(60, 60),                            # half size of the patch (final patch will be 100x100)
                stride=(40, 40),                        # overlap among patches (keep it at least large as 4 times the neuron size)
                only_init_patch=True,                   # just leave it as is
                gnb=1,                                  # number of background components
                method_deconvolution='oasis',           #could use 'cvxpy' alternatively
                low_rank_background=True,               #leave as is
                update_background_components=True,      # sometimes setting to False improve the results
                normalize_init=False,                   # just leave as is
                center_psf=True,                        # leave as is for 1 photon
                del_duplicates=True)                    # whether to remove duplicates from initialization

cnm.fit(Y)

### components evaluation

In [ ]:
idx_components, idx_components_bad, comp_SNR, r_values, pred_CNN = estimate_components_quality_auto(
                            Y, cnm.A, cnm.C, cnm.b, cnm.f, cnm.YrA, 30, 
                            0.4, cnm.gSig, dims, dview = dview, 
                            min_SNR=2, r_values_min = 0.8, min_std_reject = 1, use_cnn = False)
dview.terminate()

### save results

In [ ]:
try:
    cnmfds.close()
except NameError:
    pass
cnmfds = save_cnmf(cnm, dpath, unit_mask=idx_components, meta_dict=meta_dict, order='F')

### visualization

In [ ]:
cnmfds = xr.open_dataset(dpath + os.sep + "cnm.nc")
Yds = xr.open_dataarray(dpath + os.sep + "varr_mc_int.nc")
cnmfviewer = mvis.CNMFViewer(cnmfds, Yds)

In [ ]:
cnmfviewer.show(port=10000)